# Image Generation Agent with MCP

In [5]:
import os
from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Colab secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [6]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("ADK + MCP components imported successfully.")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


ADK + MCP components imported successfully.


In [7]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

print("Retry configuration Function created.")

Retry configuration Function created.


In [8]:
# MCP image server 1: Everything server with getTinyImage
mcp_everything_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=60,
    )
)

# MCP image server 2: Example remote server (can be replaced with another public MCP image server)
mcp_remote_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "mcp-remote",
                "https://www.kaggle.com/mcp",
            ],
        ),
        timeout=60,
    )
)

IMAGE_MCP_SERVERS = {
    "everything_demo": mcp_everything_image_server,
    "remote_demo": mcp_remote_image_server,
}

print("MCP Image Toolsets created:", list(IMAGE_MCP_SERVERS.keys()))

MCP Image Toolsets created: ['everything_demo', 'remote_demo']


In [9]:
BULK_IMAGE_THRESHOLD = 5  # > 5 images is considered bulk


def generate_images_with_approval(
    num_images: int,
    prompt: str,
    server_name: str,
    tool_context: ToolContext,
) -> dict:
    """Approval gate for MCP image generation.

    This tool does **NOT** directly call the MCP server. Instead, it:
    - Decides whether approval is needed
    - Uses ADK's confirmation mechanism for bulk requests
    - Returns a status + metadata back to the agent
    """

    # SCENARIO 1: Small request (1–5 images) → auto-approve
    if num_images <= BULK_IMAGE_THRESHOLD and num_images > 0:
        return {
            "status": "approved",
            "auto_approved": True,
            "num_images": num_images,
            "prompt": prompt,
            "server_name": server_name,
            "message": f"Auto-approved request for {num_images} image(s) using '{server_name}' server.",
        }

    if num_images <= 0:
        return {
            "status": "rejected",
            "message": "Number of images must be greater than 0.",
        }

    # SCENARIO 2: First call for bulk request (> threshold) – ask for confirmation
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=(
                f"⚠️ Bulk image request: {num_images} images with prompt: '{prompt}'.\n"
                f"Server: {server_name}. Do you want to approve this generation?"
            ),
            payload={
                "num_images": num_images,
                "prompt": prompt,
                "server_name": server_name,
            },
        )
        return {
            "status": "pending",
            "message": f"Bulk request for {num_images} images requires approval.",
        }

    # SCENARIO 3: Resumed call – we have a confirmation decision
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "auto_approved": False,
            "num_images": num_images,
            "prompt": prompt,
            "server_name": server_name,
            "message": (
                f"Bulk request approved ✔️. You may now generate {num_images} image(s) "
                f"using MCP server '{server_name}'."
            ),
        }
    else:
        return {
            "status": "rejected",
            "message": (
                f"Bulk image request was rejected ❌. No images will be generated "
                f"for prompt: '{prompt}'."
            ),
        }


print("Long-running approval tool created!")

Long-running approval tool created!


In [10]:
DEFAULT_IMAGE_SERVER_KEY = "everything_demo"

image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent_with_approval",
    instruction=(
        "You are an image generation coordinator.\n"
        "- Use the 'generate_images_with_approval' tool FIRST to check whether image "
        "generation is auto-approved or requires confirmation.\n"
        "- If the tool returns status 'approved', then call one of the MCP image tools "
        "(for example from 'everything_demo') to actually generate images.\n"
        "- If status is 'pending', inform the user that the request is waiting for human approval.\n"
        "- If status is 'rejected', explain that no images will be generated."
    ),
    tools=[
        FunctionTool(func=generate_images_with_approval),
        *IMAGE_MCP_SERVERS.values(),
    ],
)

print("Image Agent with approval + MCP tools created!")

Image Agent with approval + MCP tools created!


In [11]:
session_service = InMemorySessionService()

image_app = App(
    name="image_generation_with_approval_app",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

image_runner = Runner(
    app=image_app,
    session_service=session_service,
)

print("Resumable image app + runner created!")

Resumable image app + runner created!


/tmp/ipython-input-269716761.py:6: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),
/tmp/ipython-input-269716761.py:3: UserWarning: [EXPERIMENTAL] App: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  image_app = App(


In [12]:
def check_for_approval(events):
    """Check if events contain an approval request (adk_request_confirmation)."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

In [13]:
def create_approval_response(approval_info, approved: bool):
    """Create approval response message for ADK resumable flow."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")


print("Helper functions ready.")

Helper functions ready.


In [30]:
import asyncio

async def run_image_workflow(query: str, auto_approve: bool = True):
    """Run the image generation workflow with optional human approval."""
    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    session_id = f"image_{uuid.uuid4().hex[:8]}"

    await session_service.create_session(
        app_name="image_generation_with_approval_app",
        user_id="demo_user",
        session_id=session_id,
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # First run: agent may request confirmation
    async for event in image_runner.run_async(
        user_id="demo_user",
        session_id=session_id,
        new_message=query_content,
    ):
        events.append(event)

    approval_info = check_for_approval(events)

    if approval_info:
        print("⏸️ Pausing for approval...")
        print(f"Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # Resume with human decision
        async for event in image_runner.run_async(
            user_id="demo_user",
            session_id=session_id,
            new_message=create_approval_response(approval_info, auto_approve),
            invocation_id=approval_info["invocation_id"],
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
    else:
        # No approval needed
        print_agent_response(events)

    print(f"{'='*60}\n")


print("Image workflow function ready.")

Image workflow function ready.


In [ ]:
#Examples:
await run_image_workflow("Generate 1 tiny image of a blue cat in pixel art style")
await run_image_workflow("Generate 4 tiny images of a futuristic city at night")
await run_image_workflow("Generate 8 tiny images of a fantasy landscape in watercolor style", auto_approve=True)
await run_image_workflow("Generate 8 tiny images of a fantasy landscape in watercolor style", auto_approve=False)

## Congrulations !🎉